In [2]:
train = read.csv(file = '../input/457-engineered/train_engineered (2).csv', header = TRUE)
test = read.csv(file = '../input/457-engineered/test_engineered (2).csv', header = TRUE)
train_sub = train[train$Sales > 0, ]#only use positive Sales in training data
testOpen = (test$Open == 1)
test_sub = test[testOpen, ] #only needs to do prediction for open days
colnames(train_sub)

[1] "Store"               "DayOfWeek"           "Sales"              
 [4] "Customers"           "Open"                "Promo"              
 [7] "SchoolHoliday"       "year"                "month"              
[10] "day"                 "StoreType"           "CompetitionDistance"

In [3]:
install.packages('caret')
library(caret)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘caret’


The following object is masked from ‘package:httr’:

    progress




Model for sales

Hyper parameter pruning for nrounds and eta(learning rate)

In [10]:
tune_grid <- expand.grid(
  nrounds = c(50,100,400,800),
  eta = c(0.01, 0.05, 0.1),
  max_depth = 4,
  gamma = 0.01,
  colsample_bytree = 0.8,
  min_child_weight = 3,
  subsample = 1
)

tune_control <- caret::trainControl(
  method = "cv", # cross-validation
  number = 4, # with n folds 
  verboseIter = TRUE, # no training log
  allowParallel = TRUE # FALSE for reproducible results 
)

set.seed(123)
formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + factor(month):day + CompetitionDistance
xgb_tune <- train(
                formula,
                data = train_sub,
                trControl = tune_control,
                tuneGrid = tune_grid,
                method = "xgbTree",
                verbose = TRUE,
                metric="RMSE"
)
xgb_tune$bestTune

+ Fold1: eta=0.01, max_depth=4, gamma=0.01, colsample_bytree=0.8, min_child_weight=3, subsample=1, nrounds=800 
[15:24:47] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.01, max_depth=4, gamma=0.01, colsample_bytree=0.8, min_child_weight=3, subsample=1, nrounds=800 
+ Fold1: eta=0.05, max_depth=4, gamma=0.01, colsample_bytree=0.8, min_child_weight=3, subsample=1, nrounds=800 
[15:26:00] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.05, max_depth=4, gamma=0.01, colsample_bytree=0.8, min_child_weight=3, subsample=1, nrounds=800 
+ Fold1: eta=0.10, max_depth=4, gamma=0.01, colsample_bytree=0.8, min_child_weight=3, subsample=1, nrounds=800 
[15:27:13] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.10, max_depth=4, gamma=0.01, 

,nrounds,max_depth,eta,gamma,colsample_bytree,min_child_weight,subsample
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,800,4,0.1,0.01,0.8,3,1


nrounds = 800, eta = 0.1

 Column and Row Sampling: min child weight and colsample_bytree

In [11]:
tune_grid <- expand.grid(
  nrounds = c(800),
  eta = c(0.1),
  max_depth = 4,
  gamma = 0.01,
  colsample_bytree = c(0.4, 0.6, 0.8, 1.0),
  min_child_weight = c(1, 2, 3),
  subsample = 1
)

tune_control <- caret::trainControl(
  method = "cv", # cross-validation
  number = 4, # with n folds 
  verboseIter = TRUE, # no training log
  allowParallel = TRUE # FALSE for reproducible results 
)

set.seed(123)
formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + factor(month):day + CompetitionDistance
xgb_tune <- train(
                formula,
                data = train_sub,
                trControl = tune_control,
                tuneGrid = tune_grid,
                method = "xgbTree",
                verbose = TRUE,
                metric="RMSE"
)
xgb_tune$bestTune

+ Fold1: eta=0.1, max_depth=4, gamma=0.01, colsample_bytree=0.4, min_child_weight=1, subsample=1, nrounds=800 
[15:45:21] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.1, max_depth=4, gamma=0.01, colsample_bytree=0.4, min_child_weight=1, subsample=1, nrounds=800 
+ Fold1: eta=0.1, max_depth=4, gamma=0.01, colsample_bytree=0.4, min_child_weight=2, subsample=1, nrounds=800 
[15:46:14] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.1, max_depth=4, gamma=0.01, colsample_bytree=0.4, min_child_weight=2, subsample=1, nrounds=800 
+ Fold1: eta=0.1, max_depth=4, gamma=0.01, colsample_bytree=0.4, min_child_weight=3, subsample=1, nrounds=800 
[15:47:08] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.1, max_depth=4, gamma=0.01, colsam

Selecting tuning parameters
Fitting nrounds = 800, max_depth = 4, eta = 0.1, gamma = 0.01, colsample_bytree = 1, min_child_weight = 3, subsample = 1 on full training set
[16:39:49] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


,nrounds,max_depth,eta,gamma,colsample_bytree,min_child_weight,subsample
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,800,4,0.1,0.01,1,3,1


 Column and Row Sampling result: colsample_bytree = 1, min_child_weight = 3

Gamma

In [4]:
tune_grid <- expand.grid(
  nrounds = c(800),
  eta = c(0.1),
  max_depth = 4,
  gamma = c(0,0.01, 0.05, 0.1, 0.5, 0.7, 0.9, 1.0),
  colsample_bytree = c(1),
  min_child_weight = c(3),
  subsample = 1
)

tune_control <- caret::trainControl(
  method = "cv", # cross-validation
  number = 4, # with n folds 
  verboseIter = TRUE, # no training log
  allowParallel = TRUE # FALSE for reproducible results 
)

set.seed(123)
formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + factor(month):day + CompetitionDistance
xgb_tune <- train(
                formula,
                data = train_sub,
                trControl = tune_control,
                tuneGrid = tune_grid,
                method = "xgbTree",
                verbose = TRUE,
                metric="RMSE"
)
xgb_tune$bestTune

+ Fold1: eta=0.1, max_depth=4, gamma=0.00, colsample_bytree=1, min_child_weight=3, subsample=1, nrounds=800 
[17:26:11] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.1, max_depth=4, gamma=0.00, colsample_bytree=1, min_child_weight=3, subsample=1, nrounds=800 
+ Fold1: eta=0.1, max_depth=4, gamma=0.01, colsample_bytree=1, min_child_weight=3, subsample=1, nrounds=800 
[17:27:36] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.1, max_depth=4, gamma=0.01, colsample_bytree=1, min_child_weight=3, subsample=1, nrounds=800 
+ Fold1: eta=0.1, max_depth=4, gamma=0.05, colsample_bytree=1, min_child_weight=3, subsample=1, nrounds=800 
[17:28:59] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
- Fold1: eta=0.1, max_depth=4, gamma=0.05, colsample_bytree

Selecting tuning parameters
Fitting nrounds = 800, max_depth = 4, eta = 0.1, gamma = 0, colsample_bytree = 1, min_child_weight = 3, subsample = 1 on full training set
[18:11:10] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


,nrounds,max_depth,eta,gamma,colsample_bytree,min_child_weight,subsample
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,800,4,0.1,0,1,3,1


Gamma result: 0

final model:

In [5]:
tune_grid <- expand.grid(
  nrounds = c(800),
  eta = c(0.1),
  max_depth = 4,
  gamma = 0,
  colsample_bytree = c(1),
  min_child_weight = c(3),
  subsample = 1
)

tune_control <- caret::trainControl(
  method = "cv", # cross-validation
  number = 5, # with n folds 
  verboseIter = FALSE, # no training log
  allowParallel = TRUE # FALSE for reproducible results 
)
set.seed(123)
formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + Promo + SchoolHoliday + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Customers + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(month) + Customers:Promo + Customers:SchoolHoliday + Customers:factor(month) + Customers:day + Promo:SchoolHoliday + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + factor(month):day + CompetitionDistance
xgb_tune <- train(
                formula,
                data = train_sub,
                trControl = tune_control,
                tuneGrid = tune_grid,
                method = "xgbTree",
                verbose = TRUE,
                metric="RMSE"
)

[18:29:04] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:30:34] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:32:07] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:33:37] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:07] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:36:37] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


Model for Customers

In [6]:
# temporarily use the same grid

tune_control <- caret::trainControl(
  method = "cv", # cross-validation
  number = 5, # with n folds 
  verboseIter = FALSE, # no training log
  allowParallel = TRUE # FALSE for reproducible results 
)

set.seed(123)
formula = Customers ~ factor(Store) + factor(DayOfWeek) + Promo + SchoolHoliday + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + factor(month):day + CompetitionDistance + StoreType
xgb_tune_c <- train(
                formula,
                data = train_sub,
                trControl = tune_control,
                tuneGrid = tune_grid,
                method = "xgbTree",
                verbose = TRUE,
                metric="RMSE"
)

[18:38:31] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:01] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:41:32] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:43:03] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:33] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:04] WARNING: amalgamation/../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [7]:
test_sub$Customers = predict(xgb_tune_c, test_sub[,-c(3)])# delete open
yhat=predict(xgb_tune,newdata=test_sub[,-3])

outDat = data.frame(ID = 1:nrow(test), Sales = rep(0,nrow(test))) 
outDat$Sales[testOpen] = yhat

#Write the prediction to an output file
write.csv(outDat, file = 'XGB&XGB_0321.csv', row.names = FALSE)

In [8]:
head(outDat)

,ID,Sales
,<int>,<dbl>
1,1,6016.864
2,2,5807.615
3,3,3588.324
4,4,5206.438
5,5,9077.982
6,6,9119.171


In [9]:
xgb_tune$bestTune

,nrounds,max_depth,eta,gamma,colsample_bytree,min_child_weight,subsample
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,800,4,0.1,0,1,3,1


In [10]:
xgb_tune_c$bestTune

,nrounds,max_depth,eta,gamma,colsample_bytree,min_child_weight,subsample
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,800,4,0.1,0,1,3,1
